In [ ]:
import os
import json
from intake import open_catalog

In [ ]:
# global arguments

USER = 'charlesbluca'
REPO = 'pangeo-datastore-stac'
BRANCH = 'master'

url = f'https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}'

root = {"href": f'{url}/master/catalog.json', "rel": "root"}

In [ ]:
# generate catalog for root

cat = open_catalog('https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml')

# base catalog
d = {"stac_version": "0.9.0",
     "id": "",
     "title": "",
     "description": "",
     "links": []}

# populate catalog
d['id'] = cat.name
d['title'] = cat.description
d['links'].append(root)
d['links'].append({"href": f'{url}/master/catalog.json', "rel": "self"})

# add children links
for child in list(cat):
    d['links'].append({"href": f'{url}/master/{child}/catalog.json',
                       "rel": "child",
                       "title": cat[child].description})

# write to file
os.makedirs('master', exist_ok=True)
with open('master/catalog.json', 'w') as f:
    json.dump(d, f, indent=4)

In [ ]:
# generate catalog/collections for all other entries

entries = cat.walk(depth=5)

for item in entries:
    if entries[item]._driver == 'intake.catalog.local.YAMLFileCatalog':
        
        # base catalog
        d = {"stac_version": "0.9.0",
             "id": "",
             "title": "",
             "description": "",
             "links": []}
        
        # populate catalog
        d['id'] = entries[item].name
        d['title'] = entries[item].description
        d['links'].append(root)
        if len(item.split('.')) == 1:
            d['links'].append({"href": f'{url}/master/catalog.json', "rel": "parent"})
        else:
            parent_str = '/'.join(item.split('.')[:-1])
            d['links'].append({"href": f'{url}/master/{parent_str}/catalog.json', "rel": "parent"})
        item_str = item.replace('.', '/')
        d['links'].append({"href": f'{url}/master/{item_str}/catalog.json', "rel": "self"})
        
        # add children links
        for child in list(entries[item]):
            if entries[item][child]._driver != 'intake.catalog.local.YAMLFileCatalog':
                d['links'].append({"href": f'{url}/master/{item_str}/{child}/collection.json',
                                   "rel": "child",
                                   "title": entries[item][child].description})
            else:
                d['links'].append({"href": f'{url}/master/{item_str}/{child}/catalog.json',
                                   "rel": "child",
                                   "title": entries[item][child].description})
        
        # write to file
        os.makedirs(f'master/{item_str}', exist_ok=True)
        with open(f'master/{item_str}/catalog.json', 'w') as f:
            json.dump(d, f, indent=4)      
    else:
        
        # base collection
        d = {"stac_version": "0.9.0",
             "stac_extensions": [],
             "id": "",
             "title": "",
             "description": "",
             "keywords": [],
             "license": "proprietary",
             "providers": [],
             "extent": {"spatial": {"bbox": [[]]}, "temporal": {"interval": [[]]}},
             "links": []}
        
        # populate collection
        d['id'] = entries[item].name
        d['title'] = entries[item].description
        d['links'].append(root)
        parent_str = '/'.join(item.split('.')[:-1])
        d['links'].append({"href": f'{url}/master/{parent_str}/catalog.json', "rel": "parent"})
        item_str = item.replace('.', '/')
        d['links'].append({"href": f'{url}/master/{item_str}/catalog.json', "rel": "self"})

        # write to file
        os.makedirs(f'master/{item_str}', exist_ok=True)
        with open(f'master/{item_str}/collection.json', 'w') as f:
            json.dump(d, f, indent=4)